In [1]:
import pymongo
from pymongo import MongoClient

In [2]:
try:
    client = MongoClient('127.0.0.1', 27017)
    db = client['crawledurls'] #連結到mongoDB裡的database
    collection = db['crawledurls'] #連結到該database的table
    print("Connects to MongoDB successfully!!")
except:
    print('Cannot connect to MongoDB server!')   

Connects to MongoDB successfully!!


In [4]:
#Get contents of articles and insert them into mongoDB
#newspaper3k 版本
#現階段已可較正確的處理newspapaer3k抓取的文章，目前要著手處理無法用newspaper3k抓取的文章
import jieba
import jieba.analyse
from newspaper import Article
from urllib.request import urlopen
from collections import Counter
import time
import datetime
import hashlib
datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#datetime記得要用成全域變數，否則會出現local variable referenced before assignment 的 error         
start = time.time()
end = time.time()
elapsedTime = end - start

def getArticle(url):
    art_dict = {} #把文章內容存成一個 dictionary
    art = Article(url, language = "zh")
    try:
        art.download() #載入文章
        art.parse() #從文字上分析
    except:
        print('cannot download texts!!')
        pass
    art_text = art.text.replace('\n', '').replace('\'', '').replace('\"', '').replace("\xa0", "").replace("\u3000", "").replace("\xa06", "").replace("\xa03", "").replace("\xa02", "").replace("\xa01", "").strip()#word_cnt.most_common()[:10]                                                                                                                                                                                                #文章內文，後續replace是為了去除一些不需要的字元                                                                 
    art_url = art.url.replace("\n", "")   #原始的url  
    unixtime = int(time.time())  #標記時間戳(系統時間)   
    hash_object = hashlib.md5(art_url.encode())
    MD5code = hash_object.hexdigest()  #將url轉成MD5並且存取下來
    jieba.set_dictionary('./dict.txt.big.txt')  #結巴斷詞字庫
    words = jieba.cut(art_text, cut_all=False) 
    words = jieba.analyse.set_stop_words('./stopwords.txt')  
    words = jieba.analyse.extract_tags(art_text,10)  #利用jieba.analyse來計算文本TF-IDF關鍵詞
    #words_filtered = [word for word in cutsentence if word not in stopwords] 
    #words_filtered = str(words_filtered)
    art_dict["content"] = art_text
    art_dict["url"] = art_url
    art_dict["unix_time"] = unixtime
    art_dict["datetime"] = datetime #文章爬取時間(year-month-date hour-minute-second)
    art_dict["MD5"] = MD5code
    art_dict["TF-IDF words list"] = str(words).replace("[", "").replace("]", "")
    
    return art_dict

f = open('C:\\Users\\CF_NB.CF.000\\uncrawledurls(not crawled)\\managertodayTest.txt','rt')
lines = f.readlines()
#print(lines).replace("\n", " ")
for i in lines:
    #urlopen(i)#.decode("utf-8")
    getArticle(i)
    
#getArticle("https://www.babyou.com/opencms/channel4/Article001688.html")

Building prefix dict from C:\Users\CF_NB.CF.000\dict.txt.big.txt ...
Loading model from cache C:\Users\CF_NBC~1.000\AppData\Local\Temp\jieba.u59a467f98f36d7ead75ba056be89879f.cache
Loading model cost 2.5438077449798584 seconds.
Prefix dict has been built succesfully.


Time taken:  0.0 seconds


Building prefix dict from C:\Users\CF_NB.CF.000\dict.txt.big.txt ...
Loading model from cache C:\Users\CF_NBC~1.000\AppData\Local\Temp\jieba.u59a467f98f36d7ead75ba056be89879f.cache
Loading model cost 2.4797613620758057 seconds.
Prefix dict has been built succesfully.


Time taken:  0.0 seconds


Building prefix dict from C:\Users\CF_NB.CF.000\dict.txt.big.txt ...
Loading model from cache C:\Users\CF_NBC~1.000\AppData\Local\Temp\jieba.u59a467f98f36d7ead75ba056be89879f.cache
Loading model cost 2.7009201049804688 seconds.
Prefix dict has been built succesfully.


Time taken:  0.0 seconds


Building prefix dict from C:\Users\CF_NB.CF.000\dict.txt.big.txt ...
Loading model from cache C:\Users\CF_NBC~1.000\AppData\Local\Temp\jieba.u59a467f98f36d7ead75ba056be89879f.cache
Loading model cost 2.6478826999664307 seconds.
Prefix dict has been built succesfully.


Time taken:  0.0 seconds


Building prefix dict from C:\Users\CF_NB.CF.000\dict.txt.big.txt ...
Loading model from cache C:\Users\CF_NBC~1.000\AppData\Local\Temp\jieba.u59a467f98f36d7ead75ba056be89879f.cache
Loading model cost 2.5668232440948486 seconds.
Prefix dict has been built succesfully.


Time taken:  0.0 seconds


In [ ]:
#定位爬蟲版本
import jieba
import jieba.analyse
import requests
from urllib.request import urlopen
import urllib.parse
from bs4 import BeautifulSoup
import time
import datetime
import pickle
datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def getTechbang(url):

    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'lxml')

    articles_dict = {}
    contents = soup.select('#article-content > p')
    contents_dict = {}
    for page in range(len(contents)):
        contents_dict[page+1] = contents[page].text.replace('\n', '').replace('\'', '').replace('\"', '').replace("\xa0", "").replace("\u3000", "").replace("\xa06", "").replace("\xa03", "").replace("\xa02", "").replace("\xa01", "").strip()   
    unixtime = int(time.time())  #標記時間戳(系統時間)   
    hash_object = hashlib.md5(url.encode())
    MD5code = hash_object.hexdigest()  #將url轉成MD5並且存取下來
    jieba.set_dictionary('./dict.txt.big.txt')  #結巴斷詞字庫
    words = jieba.cut(contents_dict, cut_all=False) 
    words = jieba.analyse.set_stop_words('./stopwords.txt')  
    words = jieba.analyse.extract_tags(str(contents_dict),10)  #利用jieba.analyse來計算文本TF-IDF關鍵詞

    articles_dict['url'] = url.replace("\n", "")
    articles_dict['MD5'] = MD5code
    articles_dict['unix_time'] = unixtime
    articles_dict['datetime'] = datetime
    articles_dict["TF-IDF words list"] = str(words).replace("[", "").replace("]", "")
    articles_dict['contents'] = str(contents_dict).replace("{", "").replace("}", "").replace("1: ", "").replace("2: ", "").replace("3: ", "").replace("4: ", "").replace("5: ", "").replace("6: ", "").replace("7: ", "").replace("8: ", "").replace("9: ", "").replace("10: ", "").replace("11: ", "").replace("12: ", "").replace("13: ", "").replace("14: ", "").replace("15: ", "").replace("16: ", "").replace("17: ", "").replace("18: ", "").replace("19: ", "").replace("20: ", "").replace("21: ", "").replace("22: ", "").replace("23: ", "").replace("24: ", "").replace("25: ", "").replace("26: ", "").replace("'',", " ").replace("1'", "").replace("1 ", "").replace("\\t", "").replace("\\r", "")
    #print(articles_dict)

    return articles_dict

#
f = open('C:\\Users\\CF_NB.CF.000\\uncrawledurls(not crawled)\\techbangTest.txt','rt')
lines = f.readlines()
#print(lines).replace("\n", " ")
for i in lines:
    #urlopen(i)#.decode("utf-8")
    getTechbang(i)

#getTechbang('https://www.techbang.com/posts/52078-a-recovered-lost-samsung-september-out-of-the-most-expensive-mobile-phones-in-the-history-of-its-galaxy-note8-priced-at-nearly-1000-euros')

In [ ]:
#newspaper3k版本的爬蟲輸入mongoDB
f = open('C:\\Users\\CF_NB.CF.000\\uncrawledurls(k)\\allcombined(k).txt','rt')
lines = f.readlines()

#for i in lines:
    #urlopen(i)
    #getArticle(i) 
for i in lines:
    art_dict = getArticle(i)
    try:
        collection.insert_one(art_dict)
    except:
        pass
    
#-----------------------------------------------------------------------------------------#

#for i in lines:    
    #art_dict = getArticle(i)    
    #print('%s has %s paras'%(art_dict['title'],len(art_dict['content'])))    
    #if(len(art_dict["content"]) > 100): #字數小於100字的文章不存
        #collection.insert_one(art_dict)
        #print('[Insert %s paras ok!] %s'%(len(art_dict['content']),art_dict['title']))
    #else:
        #pass
    #c.execute("""UPDATE cklist SET tag = 'R' WHERE url = %s""", url) #這是原本要用在取MySQL裏頭的urls，在此先不需要用到

In [ ]:
#定位爬蟲版本的輸入mongoDB
f = open('C:\\Users\\CF_NB.CF.000\\uncrawledurls(old)\\beautimode(test).txt','rt')
lines = f.readlines()

#for i in lines:
    #urlopen(i)
    #getArticle(i)  
for i in lines:    
    art_dict = getBeautimode(i)    
    #print('%s has %s paras'%(art_dict['title'],len(art_dict['content'])))    
    #if(len(art_dict['content']) > 3): #字數小於100字的文章不存
        #collection.insert_one(art_dict)
        #print('[Insert %s paras ok!] %s'%(len(art_dict['content']),art_dict['title']))
    #else:
        #pass
    #c.execute("""UPDATE cklist SET tag = 'R' WHERE url = %s""", url) #這是原本要用在取MySQL裏頭的urls，在此先不需要用到
    try:
        collection.insert_one(art_dict)
    except:
        pass

In [ ]:
if 